In [ ]:
import cv2
import tensorflow
import numpy as np
from ultralytics import YOLO

In [ ]:
# Load the YOLOv8 model
model = YOLO('V:/coding/dev/Python/Gantry/FYP/best.pt')

# Function to perform detection on an image
def detect_image(image_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")

    # Perform inference
    results = model(image)

    # Extract results
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes
    scores = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)  # Class IDs

    # Draw the results on the image
    for box, score, class_id in zip(boxes, scores, class_ids):
        if score > 0.2:  # Threshold for displaying boxes
            x1, y1, x2, y2 = map(int, box[:4])
            label = f"Class {class_id}: {score:.2f}"
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the output image
    cv2.imwrite(output_path, image)
    print(f"Output image saved at: {output_path}")



In [ ]:
input_image_path = 'V:/coding/dev/Python/Gantry/FYP/raw/resized/20241006_172958(1).jpg'
output_image_path = 'V:/coding/dev/Python/Gantry/FYP/output/20241006_172958(1)_detected.jpg'
#output_image_path = './output/20241006_125620_detected.jpg'
detect_image(input_image_path, output_image_path)


0: 640x640 3 Healthys, 3 Black Rot Diseases, 129.0ms
Speed: 4.0ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Output image saved at: V:/coding/dev/Python/Gantry/FYP/output/20241006_172958(1)_detected.jpg


In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="V:/coding/dev/Python/Gantry/FYP/best_float32.tflite")
interpreter.allocate_tensors()

#input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
image_path = "V:/coding/dev/Python/Gantry/FYP/resized/20241006_131544.jpg"
image = cv2.imread(image_path)
input_shape = input_details[0]['shape']
input_data = cv2.resize(image, (input_shape[1], input_shape[2]))
input_data = np.expand_dims(input_data, axis=0).astype(np.float32) / 255.0

# inference
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Extract and process predictions
detections = interpreter.get_tensor(output_details[0]['index'])

# Process the predictions
for detection in detections[0]:  # Iterate through detected objects
    confidence = detection[4]
    if confidence > 0.1:  # Confidence threshold
        xmin, ymin, xmax, ymax = detection[0:4]
        (left, right, top, bottom) = (int(xmin * image.shape[1]), int(xmax * image.shape[1]),
                                      int(ymin * image.shape[0]), int(ymax * image.shape[0]))
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        class_id = int(detection[5])
        label = f"Class {class_id}: {confidence:.2f}"
        cv2.putText(image, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display and save the image with detections
cv2.imshow("Detected Objects", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("V:/coding/dev/Python/Gantry/FYP/output/16output_image.jpg", image)

True